### Lexical Substitution Effect estimation using CausalForest
- R package: https://cran.r-project.org/web/packages/SparseM/vignettes/SparseM.pdf <br>
- n_trees = [200, 2000] <br>

In [1]:
import requests, time, operator, re, json, csv, pickle, copy, gc, ast
import timeit
import pandas as pd
import numpy as np
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from ast import literal_eval
from scipy import io
from scipy.sparse import vstack
from scipy.spatial.distance import cdist
import sklearn.metrics.pairwise
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import r
from rpy2.robjects import Matrix
import rpy2.robjects.numpy2ri
from rpy2.robjects.vectors import Matrix
from rpy2.robjects.packages import importr
importr('grf')
rpy2.robjects.numpy2ri.activate()
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

AMT_path = '/data/2/zwang/2018_S_WordTreatment/V2_AMT/'
airbnb_path = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/'
tw_path = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/'
yp_path = '/data/2/zwang/2018_S_WordTreatment/V2_yelp/'

ModuleNotFoundError: No module named 'rpy2'

In [2]:
def CSF10_treatment_for_pair(sents,labels,divide,CSF_pred):
    source_pos_treatment = []
    source_neg_treatment = []
    target_pos_treatment = []
    target_neg_treatment = []
    
    for i in range(len(sents)):
        if(i<divide and labels[i]==1):
            source_pos_treatment.append((sents[i],round(float(CSF_pred[i]),5)))#,CSF_var[i]
        elif(i<divide and labels[i]==0):
            source_neg_treatment.append((sents[i],round(float(CSF_pred[i]),5)))#,CSF_var[i]
        elif(i>=divide and labels[i]==1):
            target_pos_treatment.append((sents[i],round(float(CSF_pred[i]),5)))#,CSF_var[i]
        elif(i>=divide and labels[i]==0):
            target_neg_treatment.append((sents[i],round(float(CSF_pred[i]),5)))#,CSF_var[i]
            
    return {'source_pos_sents_treatment':[source_pos_treatment],'source_neg_sents_treatment':[source_neg_treatment],
            'target_pos_sents_treatment':[target_pos_treatment],'target_neg_sents_treatment':[target_neg_treatment]}

In [3]:
def train_10CSF(X_src,y_src,W_src,X_src_testidx,X_src_trainidx,X_tar,y_tar,W_tar,X_tar_testidx,X_tar_trainidx,nfold):
    #https://cran.r-project.org/web/packages/SparseM/vignettes/SparseM.pdf
    
    # train 10 CF for 10 folds of X_src
    src_pred = []
    #src_var = []
    tar_pred = []
    #tar_var = []
    
    for i in range(X_src.shape[0]):
        src_pred.append(-2)
        #src_var.append(-2)
    for i in range(X_tar.shape[0]):
        tar_pred.append(-2)
        #tar_var.append(-2)
    
    X_src_arr = X_src.toarray()
    X_tar_arr = X_tar.toarray()
    y_src_arr = np.array(y_src)
    y_tar_arr = np.array(y_tar)
    W_src_arr = np.array(W_src)
    W_tar_arr = np.array(W_tar)
    
    
    for i in range(nfold):
        test_vec_src = X_src_arr[X_src_testidx[i]]
        #test_vec_src_arr = test_vec_src.toarray()
        X_test_src = r.matrix(test_vec_src, nrow=test_vec_src.shape[0], ncol=test_vec_src.shape[1])
        #X_test_src = r.matrix(test_vec_src, nrow=test_vec_src.shape[0], ncol=test_vec_src.shape[1],sparce=True)
        train_vec_src = X_src_arr[X_src_trainidx[i]]
        train_y_src = y_src_arr[X_src_trainidx[i]]
        train_W_src = W_src_arr[X_src_trainidx[i]]
        
        test_vec_tar = X_tar_arr[X_tar_testidx[i]]
        #test_vec_tar_arr = test_vec_tar.toarray()
        X_test_tar = r.matrix(test_vec_tar, nrow=test_vec_tar.shape[0], ncol=test_vec_tar.shape[1])
        train_vec_tar = X_tar_arr[X_tar_trainidx[i]]
        train_y_tar = y_tar_arr[X_tar_trainidx[i]]
        train_W_tar = W_tar_arr[X_tar_trainidx[i]]
        
        train_vec_all = np.vstack((train_vec_src,train_vec_tar))
        #train_vec_all_arr = train_vec_all.toarray()
        X_train = r.matrix(train_vec_all, nrow=train_vec_all.shape[0], ncol=train_vec_all.shape[1])   
        y_train = list(train_y_src) + list(train_y_tar)
        W_src_train = list(train_W_src) + list(train_W_tar)
        W_tar_train = list(1-np.array(W_src_train))
        
        r.assign('X',X_train)
        #r.assign('X',train_vec_all)
        r.assign('Y',ro.FloatVector(y_train))
        r.assign('W.src',ro.IntVector(W_src_train))
        r.assign('W.tar',ro.IntVector(W_tar_train))
        r.assign('X.test_src',X_test_src)
        r.assign('X.test_tar',X_test_tar)
        
        # honesty = FALSE, ci.group.size=1, sample.fraction=0.8
        r('c.src_forest <- causal_forest(X,Y,W.src, num.trees=2000, num.threads=8, seed=set.seed(10))')
        r('c.pred_src <- predict(c.src_forest, X.test_src, num.threads=8)')
        
        r('c.tar_forest <- causal_forest(X,Y,W.tar, num.trees=2000, num.threads=8, seed=set.seed(10))')
        r('c.pred_tar <- predict(c.tar_forest, X.test_tar, num.threads=8)')
        
        CSF_pred_src = list(r('c.pred_src')[0])
        #CSF_var_src = list(r('c.pred_src')[1])
        CSF_pred_tar = list(r('c.pred_tar')[0])
        #CSF_var_tar = list(r('c.pred_tar')[1])
        
        
        j=0
        for idx in X_src_testidx[i]:
            src_pred[idx] = CSF_pred_src[j]
            #src_var[idx] = CSF_var_src[j]
            j += 1
            
        k=0
        for idx in X_tar_testidx[i]:
            tar_pred[idx] = CSF_pred_tar[k]
            #tar_var[idx] = CSF_var_tar[k]
            k += 1
       
                
    return src_pred,tar_pred

In [4]:
def make_folds(X_sents,labels,divide,nfold):
    # split X_src into 10 folds, and split X_tar into 10 folds
    X_src = X_sents[:divide]
    X_tar = X_sents[divide:]
    y_src = labels[:divide]
    y_tar = labels[divide:]
    
    X_src_testidx = []
    X_src_trainidx = []
    X_tar_testidx = []
    X_tar_trainidx = []
    skf = StratifiedKFold(n_splits=nfold,random_state=42)
    for train_index, test_index in skf.split(X_src, y_src):
        X_src_testidx.append(test_index)
        X_src_trainidx.append(train_index)
    
    for train_index, test_index in skf.split(X_tar, y_tar):
        X_tar_testidx.append(test_index)
        X_tar_trainidx.append(train_index)
    
    W_asg = list(np.zeros(divide))+list(np.ones(X_sents.shape[0]-divide))
    
    src_pred, tar_pred = train_10CSF(X_src,y_src,W_asg[:divide],X_src_testidx,X_src_trainidx,
                                                       X_tar,y_tar,W_asg[divide:],X_tar_testidx,X_tar_trainidx,
                                                       nfold)
    
    return list(src_pred)+list(tar_pred) #, list(src_var)+list(tar_var)

In [5]:
def cal_10CSF_treatment(wdpair_file,treat_file,log_file,vocab_file,nfold):
    wdpairs_pos_neg_pd = pd.DataFrame(pickle.load(open(wdpair_file,'rb')))
    
    if(vocab_file):
        my_vocab = list(set(pd.read_csv(vocab_file).word.values))
        ct_vec = CountVectorizer(min_df=3,binary=True,vocabulary=my_vocab)
        #print(len(my_vocab))
    else:
        ct_vec = CountVectorizer(min_df=3,binary=True)
    
    with open(treat_file,'wt') as fw, open(log_file,'wt') as flog:
        my_fields = ['source','target','source_pos_sents_treatment','source_neg_sents_treatment','target_pos_sents_treatment','target_neg_sents_treatment']
        csv_f = csv.DictWriter(fw,fieldnames=my_fields)
        csv_f.writeheader()
            
        log_fields = ['swd','twd','shape']
        log_csv = csv.DictWriter(flog,fieldnames=log_fields)
        log_csv.writeheader()

        for idx, row in wdpairs_pos_neg_pd.iterrows():
            if(idx %100 == 0):
                print("-------------",idx,"-------------")
                
            swd = row['source']
            twd = row['target']
            swd_pos_sents = row['source_pos_sents']
            swd_neg_sents = row['source_neg_sents']
            twd_pos_sents = row['target_pos_sents']
            twd_neg_sents = row['target_neg_sents']
            
            #if((len(swd_pos_sents)>nfold) and (len(swd_neg_sents)>nfold) and (len(twd_pos_sents)>nfold) and (len(twd_neg_sents)>nfold)):
            sents = swd_pos_sents + swd_neg_sents + twd_pos_sents + twd_neg_sents
            labels = list(np.ones(len(swd_pos_sents)))+list(np.zeros(len(swd_neg_sents)))+list(np.ones(len(twd_pos_sents)))+list(np.zeros(len(twd_neg_sents)))
            divide = len(swd_pos_sents)+len(swd_neg_sents)
            X_sents = ct_vec.fit_transform(sents)
            vocab = ct_vec.vocabulary_
            swdi = vocab[swd]
            twdi = vocab[twd]

            X_sents[:,swdi] = 0
            X_sents[:,twdi] = 0

            #print(swd,twd,X_sents.shape)

            log_csv.writerow({'swd':swd, 'twd':twd, 'shape': X_sents.shape})

            CSF_pred = make_folds(X_sents,labels,divide,nfold)

            treat_info = CSF10_treatment_for_pair(sents,labels,divide,CSF_pred)

            if(treat_info):
                csv_f.writerow({'source':swd,'target':twd,
                        'source_pos_sents_treatment':treat_info['source_pos_sents_treatment'],'source_neg_sents_treatment':treat_info['source_neg_sents_treatment'],
                            'target_pos_sents_treatment':treat_info['target_pos_sents_treatment'],'target_neg_sents_treatment':treat_info['target_neg_sents_treatment']})



- Only using AMT labeled data

In [6]:
prefix = "yp"
pair_file = AMT_path+'AMT_WdSents/Data/'+prefix+'_AMT_wdsents_markPPN.pickle'
treat_file = AMT_path +"AMT_WdSents/3_CSF/"+prefix+"_csf_2000tree_treatment.csv"
log_file =  AMT_path+"AMT_WdSents/3_CSF/"+prefix+"_csf_2000tree_log.csv"

#vocab_file=airbnb_path+'0_Data/common_wds.csv'
start = time.time()
cal_10CSF_treatment(pair_file,treat_file,log_file,vocab_file='',nfold=10)
end = time.time()
print((end-start)/60)

------------- 0 -------------
68.06570002237956


In [10]:
res_pd = pd.read_csv(AMT_path +"AMT_WdSents/3_CSF/yp_csf_param_treatment_0.csv",index_col=False)
res_pd.shape

(18, 6)

In [13]:
prefix = "tw"
pair_file = AMT_path+'AMT_WdSents/Data/'+prefix+'_AMT_wdsents_markPPN.pickle'
treat_file = AMT_path +"AMT_WdSents/3_CSF/"+prefix+"_csf_param_treatment.csv"
log_file =  AMT_path+"AMT_WdSents/3_CSF/"+prefix+"_csf_param_log.csv"

#vocab_file=airbnb_path+'0_Data/common_wds.csv'
start = time.time()
cal_10CSF_treatment(pair_file,treat_file,log_file,vocab_file='',nfold=10)
end = time.time()
print((end-start)/60)

------------- 0 -------------
36.9123366912206


In [14]:
prefix = "airbnb"
pair_file = AMT_path+'AMT_WdSents/Data/'+prefix+'_AMT_wdsents_markPPN.pickle'
treat_file = AMT_path +"AMT_WdSents/3_CSF/"+prefix+"_csf_param_treatment.csv"
log_file =  AMT_path+"AMT_WdSents/3_CSF/"+prefix+"_csf_param_log.csv"

#vocab_file=airbnb_path+'0_Data/common_wds.csv'
start = time.time()
cal_10CSF_treatment(pair_file,treat_file,log_file,vocab_file='',nfold=10)
end = time.time()
print((end-start)/60)

------------- 0 -------------
99.13416692813237


In [8]:
this_path = airbnb_path
prefix = "airbnb"
pair_file = this_path+"1_Process/"+prefix+"_wdpair_sents_limit5000.pickle"
treat_file = this_path +"3_CausalForest/"+prefix+"_csf_200tree_treatment.csv"
log_file =  this_path+"3_CausalForest/"+prefix+"_csf_200tree_log.csv"
vocab_file = this_path+'0_Data/common_wds.csv'
start = time.time()
cal_10CSF_treatment(pair_file,vocab_file,treat_file,log_file,nfold=10)
end = time.time()
print((end-start)/60)

------------- 5 -------------
7.753159054120382


In [9]:
this_path = tw_path
prefix = "tw"
pair_file = this_path+"1_Process/"+prefix+"_wdpair_sents_limit5000.pickle"
treat_file = this_path +"3_CausalForest/"+prefix+"_csf_200tree_treatment.csv"
log_file =  this_path+"3_CausalForest/"+prefix+"_csf_200tree_log.csv"
start = time.time()
cal_10CSF_treatment(pair_file,treat_file,log_file,nfold=10)
end = time.time()
print((end-start)/60)

------------- 5 -------------
6.3959188540776575


In [15]:
this_path = yp_path
prefix = "yp"
pair_file = this_path+"1_Process/"+prefix+"_wdpair_sents_limit5000.pickle"
treat_file = this_path +"3_CausalForest/"+prefix+"_csf_200tree_treatment.csv"
log_file =  this_path+"3_CausalForest/"+prefix+"_csf_200tree_log.csv"
start = time.time()
cal_10CSF_treatment(pair_file,treat_file,log_file,nfold=10)
end = time.time()
print((end-start)/60)

------------- 0 -------------
------------- 100 -------------
------------- 200 -------------
------------- 300 -------------
------------- 400 -------------
------------- 500 -------------
------------- 600 -------------
------------- 700 -------------
------------- 800 -------------
681.4869681795439
